In [ ]:
# Install necessary libraries (updated for FAISS)
!uv pip install scikit-learn matplotlib torch transformers sentence-transformers mteb faiss-cpu langchain langchain-community psutil

Streaming output truncated to the last 5000 lines.
mteb       ------------------------------ 16.00 KiB/1.97 MiB
langchain-community ------------------------------ 16.00 KiB/2.41 MiB
nvidia-cuda-cupti-cu12 ------------------------------ 99.45 KiB/13.17 MiB
nvidia-nvjitlink-cu12 ------------------------------ 526.72 KiB/20.09 MiB
nvidia-cuda-nvrtc-cu12 ------------------------------ 462.06 KiB/23.50 MiB
faiss-cpu  ------------------------------ 456.56 KiB/29.25 MiB
nvidia-curand-cu12 ------------------------------ 492.34 KiB/53.70 MiB
nvidia-cusolver-cu12 ------------------------------ 576.53 KiB/122.01 MiB
nvidia-cusparse-cu12 ------------------------------ 603.39 KiB/197.84 MiB
nvidia-cufft-cu12 ------------------------------ 653.86 KiB/201.66 MiB
nvidia-cublas-cu12 ------------------------------ 477.11 KiB/346.60 MiB
⠸ Preparing packages... (6/27)
pydantic-settings ------------------------------ 43.32 KiB/43.32 KiB
dill       ------------------------------ 78.93 KiB/113.53 KiB
multipr

In [ ]:
# Import libraries (updated vector store import)
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import ndcg_score
import nltk
import re
import torch
from sentence_transformers import SentenceTransformer
from mteb import MTEB
import pandas as pd
from langchain.docstore.document import Document
from langchain.vectorstores import FAISS  # Changed import
from langchain.embeddings import HuggingFaceEmbeddings  # Updated embedding class
import time
import psutil
import shutil

# Download NLTK resources (unchanged)
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# Text cleaning function (unchanged)
def simple_clean(text):
    text = text.lower()
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [ ]:
# Data loading (unchanged)
df = pd.read_excel('/content/synthetic_resume_summaries (1).xlsx')
df.head()

,text,summary
0,Medical Advisor - General Medicines (Diabetes)...,**Medical Advisor** with over 15 years of expe...
1,"linkedin.com/in/hafsah09/\n\nSUMMARY ,\n* Over...",**Digital Product Management Professional** wi...
2,"'o\n\n+\n\nSummary ,\nI have a very positive a...",**Accountant** with experience in financial ma...
3,BRIDGETTE\nWENG\n2007\n\nPROFILE\n\nExtremely ...,**Medical Assistant** with hands-on experience...
4,"Professional Summary ,\nA highly organised, mo...",**Aspiring Legal Professional** with a strong ...


In [ ]:
texts = df["text"].tolist()
summaries = df["summary"].tolist()
N = len(texts)

# Data cleaning (unchanged)
texts_clean = [simple_clean(text) for text in texts]
summaries_clean = [simple_clean(str(summary)) for summary in summaries]

# Document creation (unchanged)
docs = [Document(page_content=text, metadata={"id": i}) for i, text in enumerate(texts_clean)]

In [ ]:
# Embedding setup (updated to HuggingFaceEmbeddings)
model_name = "multi-qa-mpnet-base-dot-v1"
embedding_function = HuggingFaceEmbeddings(model_name=model_name)

<ipython-input-6-58611bba2be3>:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = HuggingFaceEmbeddings(model_name=model_name)
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.war

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/8.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# FAISS Ingestion with persistence (modified section)
start_time = time.time()
vectorstore = FAISS.from_documents(
    documents=docs,
    embedding=embedding_function
)
# Save FAISS index to disk
vectorstore.save_local("faiss_index")
ingestion_time = time.time() - start_time
print(f"Ingestion time: {ingestion_time} seconds")

Ingestion time: 35.73035407066345 seconds


In [ ]:
# Retriever setup (unchanged interface)
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

In [ ]:
# Performance metrics (unchanged collection logic)
latencies = []
recalls = []

for i, query in enumerate(summaries_clean):
    start_time = time.time()
    retrieved_docs = retriever.get_relevant_documents(query)
    latency = time.time() - start_time
    latencies.append(latency)

    retrieved_ids = [doc.metadata["id"] for doc in retrieved_docs]
    correct_id = i
    recalls.append(1 if correct_id in retrieved_ids else 0)

# Calculate metrics (unchanged)
average_recall = sum(recalls) / len(recalls)
print(f"Average recall@1: {average_recall}")

<ipython-input-9-94dea75f0b51>:7: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(query)


Average recall@1: 0.703


In [ ]:
average_latency = sum(latencies) / len(latencies)
print(f"Average latency: {average_latency} seconds")

Average latency: 0.015947448968887327 seconds


In [ ]:
total_queries = len(summaries_clean)
total_time = sum(latencies)
QPS = total_queries / total_time if total_time > 0 else 0
print(f"QPS: {QPS}")

QPS: 62.705953908424455


In [ ]:
# Memory usage (unchanged)
memory_usage = psutil.virtual_memory().used / (1024 ** 3)
print(f"Memory usage: {memory_usage} GB")

Memory usage: 2.261425018310547 GB


In [ ]:
# Disk usage measurement updated for FAISS
disk_usage = shutil.disk_usage('./faiss_index').used / (1024 ** 3)
print(f"Disk usage for faiss_index: {disk_usage} GB")

Disk usage for faiss_index: 40.1653938293457 GB
